In [ ]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
using Random
using Distributions
using Roots
using GlobalSensitivity
using Plots
using LaTeXStrings
using Measures

# Review and Questions

## Robustness

- **Robustness**: How well does your solution work under alternative model specifications/parameter choices?
- Common measures: regret, satisfycing

# Sensitivity Analysis

## Factor Prioritization

Many parts of a systems-analysis workflow involve potentially large numbers of modeling assumptions, or **factors**:

- Model parameters/structures
- Forcing scenarios/distributions
- Tuning parameters (*e.g.* for simulation-optimization)

Additional factors increase computational expense and analytic complexity.

## Prioritizing Factors Of Interest

:::: {.columns}
::: {.column width=40%}
- How do we know which factors are most relevant to a particular analysis?
- What modeling assumptions were most responsible for output uncertainty?

:::
::: {.column width=60%}
![Sensitivity Analysis Schematic](https://ars.els-cdn.com/content/image/1-s2.0-S1364815218302822-gr1_lrg.jpg)

::: {.caption}
Source: [Saltelli et al (2019)](https://doi.org/10.1016/j.envsoft.2019.01.012)
:::
:::

::::

## Sensitivity Analysis

::: {.fragment .fade-in}
Sensitivity analysis is...

::: {.quote}
the study of how uncertainty in the output of a model (numerical or otherwise) can be apportioned to different sources of uncertainty in the model input

::: {.cite}
--- Saltelli et al (2004), *Sensitivity Analysis in Practice*
:::
:::
:::

## Why Perform Sensitivity Analysis?

:::: {.columns}
::: {.column width=50%}

:::
::: {.column width=50%}
![Sensitivity analysis modes](https://uc-ebook.org/docs/html/_images/figure3_2_factor_mapping.png)

::: {.caption}
Source: [Reed et al (2022)](https://uc-ebook.org/)
:::
:::
::::

## Factor Prioritization

:::: {.columns}
::: {.column width=50%}
Which factors have the greatest impact on output variability?

:::
::: {.column width=50%}
![Sensitivity analysis modes](https://uc-ebook.org/docs/html/_images/figure3_2_factor_mapping.png)

::: {.caption}
Source: [Reed et al (2022)](https://uc-ebook.org/)
:::
:::
::::

## Factor Fixing

:::: {.columns}
::: {.column width=50%}
Which factors have negligible impact and can be fixed in subsequent analyses?

:::
::: {.column width=50%}
![Sensitivity analysis modes](https://uc-ebook.org/docs/html/_images/figure3_2_factor_mapping.png)

::: {.caption}
Source: [Reed et al (2022)](https://uc-ebook.org/)
:::
:::
::::

## Factor Mapping

:::: {.columns}
::: {.column width=50%}
Which values of factors lead to model outputs in a certain output range?

:::
::: {.column width=50%}
![Sensitivity analysis modes](https://uc-ebook.org/docs/html/_images/figure3_2_factor_mapping.png)

::: {.caption}
Source: [Reed et al (2022)](https://uc-ebook.org/)
:::
:::
::::

## Example: Shadow Prices Are Sensitivities!

We've seen one example of a quantified sensitivity before: **the shadow price of an LP constraint.**

The shadow price expresses the objective's sensitivity to a unit relaxation of the constraint.

## Shadow Prices As Sensitivities


In [ ]:
x1 = 0:1200
x2 = 0:1400
f1(x) = (600 .- 0.9 .* x) ./ 0.5
f2(x) = 1000 .- x
f3(x) = (650 .- 0.9 .* x) ./ 0.5

p = plot(0:667, min.(f1(0:667), f2(0:667)), fillrange=0, color=:lightblue, grid=true, label="Feasible Region", xlabel=L"x_1", ylabel=L"x_2", xlims=(-50, 1200), ylims=(-50, 1400), framestyle=:origin, minorticks=4, right_margin=4mm, left_margin=4mm, legendfontsize=14, tickfontsize=16, guidefontsize=16)
plot!(0:667, f1.(0:667), color=:brown, linewidth=3, label=false)
plot!(0:1000, f2.(0:1000), color=:red, linewidth=3, label=false)
annotate!(400, 1100, text(L"0.9x_1 + 0.5x_2 = 600", color=:purple, pointsize=18))
annotate!(1000, 300, text(L"x_1 + x_2 = 1000", color=:red, pointsize=18))
plot!(size=(1200, 600))

Z(x1,x2) = 230 * x1 + 120 * x2
contour!(0:660,0:1000,(x1,x2)->Z(x1,x2), levels=5, c=:devon, linewidth=2, colorbar = false, clabels = true) 
scatter!(p, [0, 0, 667, 250], [0, 1000, 0, 750], markersize=10, z=2, label="Corner Point", markercolor=:orange)
plot!(0:667, f3.(0:667), color=:brown, linewidth=3, label=false, linestyle=:dash)

## Shadow Prices As Sensitivities

::: {.center}
**Sorting by Shadow Price** &#8644; **Factor Prioritization**
:::

# Types of Sensitivity Analysis

## Categories of Sensitivity Analysis

- One-at-a-Time vs. All-At-A-Time Sampling
- Local vs. Global

## One-At-A-Time SA

**Assumption**: Factors are linearly independent (no interactions).

**Benefits**: Easy to implement and interpret.

**Limits**: Ignores potential interactions.

## All-At-A-Time SA

Number of different sampling strategies: full factorial, Latin hypercubes, more.

**Benefits**: Can capture interactions between factors.

**Challenges**: Can be computationally expensive, does not reveal where key sensitivities occur.

## Local SA

:::: {.columns}
::: {.column width=55%}
**Local** sensitivities: Pointwise perturbations from some baseline point.

Challenge: *Which point to use?*
:::
::: {.column width=45%}
![Local vs. Global Sensitivity Analysis](figures/globallocalsa.png){width=50%}
:::
::::

## Global SA

:::: {.columns}
::: {.column width=55%}
**Global** sensitivities: Sample throughout the space.

Challenge: *How to measure global sensitivity to a particular output?*

Advantage: *Can estimate interactions between parameters*
:::
::: {.column width=45%}
![Global Sensitivity Analysis](figures/globalsa.png){width=80%}
:::
::::

## How To Calculate Sensitivities?

Number of approaches. Some examples:

- Derivative-based or Elementary Effect (*Method of Morris*)
- Regression 
- Variance Decomposition or ANOVA (*Sobol Method*)
- Density-based (*$\delta$ Moment-Independent Method*)

# Example: Lake Problem

## Parameter Ranges

For a fixed release strategy, look at how different parameters influence reliability.

:::: {.columns}
::: {.column width=40%}
Take $a_t=0.03$, and look at the following parameters within ranges:
:::
::: {.column width=60%}
Parameter | Range 
:-----: | :-----:
$q$ | $(2, 3)$
$b$ | $(0.3, 0.5)$
$ymean$ | $(\log(0.01), \log(0.07))$ 
$ystd$ | $(0.01, 0.25)$
:::
::::

## Method of Morris

The Method of Morris is an *elementary effects* method.

This is a global, one-at-a-time method which averages effects of perturbations at different values $\bar{x}_i$:

$$S_i = \frac{1}{r} \sum_{j=1}^r \frac{f(\bar{x}^j_1, \ldots, \bar{x}^j_i + \Delta_i, \bar{x}^j_n) - f(\bar{x}^j_1, \ldots, \bar{x}^j_i, \ldots, \bar{x}^j_n)}{\Delta_i}$$

where $\Delta_i$ is the step size.

## Method of Morris


In [ ]:
function lake(a, y, q, b, T)
    X = zeros(T+1, size(y, 2))
    # calculate states

    for t = 1:T
        X[t+1, :] = X[t, :] .+ a[t] .+ y[t, :] .+ (X[t, :].^q./(1 .+ X[t, :].^q)) .- b.*X[t, :]
    end
    return X
end

function lake_sens(params)
    Random.seed!(1)
    T = 100
    nsamp = 1000

    q = params[1]
    b = params[2]
    ymean = params[3]
    ystd = params[4]

    lnorm = LogNormal(ymean, ystd)
    y = rand(lnorm, (T, nsamp))
    crit(x) = (x^q/(1+x^q)) - b*x
    Xcrit = find_zero(crit, 0.5)

    X = lake(0.03ones(T), y, q, b, T)
    rel = sum(X[T+1, :] .<= Xcrit) / nsamp
    return rel
end

Random.seed!(1)
s = gsa(lake_sens, Morris(), 
    [(2, 3), (0.3, 0.5), (log(0.01), log(0.07)), 
    (0.01, 0.25)])

p1 = bar([L"$q$", L"$b$", "ymean", "ystd"], (abs.(s.means) .+ 0.01)', legend=false, title="Sensitivity Index Means", guidefontsize=18, titlefontsize=22, tickfontsize=14)
p2 = bar([L"$q$", L"$b$", "ymean", "ystd"], (s.variances .+ 0.01)', legend=false, yaxis=:log, title="Sensitivity Index Variances", guidefontsize=18, titlefontsize=22, tickfontsize=14)
plot(p1, p2, layout=(2,1), size=(1200, 600))

## Sobol' Method

The Sobol method is a variance decomposition method, which attributes the variance of the output into contributions from individual parameters or interactions between parameters.

$$S_i^1 = \frac{Var_{x_i}\left[E_{x_{\sim i}}(x_i)\right]}{Var(y)}$$

$$S_{i,j}^2 = \frac{Var_{x_{i,j}}\left[E_{x_{\sim i,j}}(x_i, x_j)\right]}{Var(y)}$$

## Sobol' Method: First and Total Order


In [ ]:
s = gsa(lake_sens, Sobol(order=[0, 1, 2], nboot=10), 
    [(2, 3), (0.3, 0.5), (log(0.01), log(0.07)), (0.01, 0.25)];
    samples = 100)
p1 = bar([L"$q$", L"$b$", "ymean", "ystd"], (abs.(s.S1) .+ 0.01), legend=false, title="First-Order Sensitivity Index", guidefontsize=18, titlefontsize=22, tickfontsize=16)
p2 = bar([L"$q$", L"$b$", "ymean", "ystd"], (s.ST .+ 0.01), legend=false, yaxis=:log, title="Sensitivity Index Variances", guidefontsize=18, titlefontsize=22, tickfontsize=16, ylimits=(0, 1))

## Sobol' Method: Second Order


In [ ]:
heatmap([L"$q$", L"$b$", "ymean", "ystd"], [L"$q$", L"$b$", "ymean", "ystd"], s.S2, , guidefontsize=18, titlefontsize=22, tickfontsize=16, title="Second Order Sensitivity Indices")

## Example: Cumulative CO~2~ Emissions

![Model for CO2 Emissions](figures/co2-model.png)

## Example: Cumulative CO~2~ Emissions

![CO2 Emissions Sensitivities](https://media.springernature.com/full/springer-static/image/art%3A10.1007%2Fs10584-021-03279-7/MediaObjects/10584_2021_3279_Fig5_HTML.png)

::: {.caption}
Source: [Srikrishnan et al (2022)](https://doi.org/10.1007/s10584-021-03279-7)
:::

# Key Takeaways

## Key Takeaways

- Sensitivity Analysis involves attributing variability in outputs to input factors.
- Factor prioritization, factor fixing, factor mapping are key modes.
- Different types of sensitivity analysis: choose carefully based on goals, computational expense, input-output assumptions.
- Many resources for more details.

# Upcoming Schedule

## Next Classes

**Monday**: Multiple Objectives and Tradeoffs

## Assessments

**Today**: Lab 4 Due